In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
import time
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
import mysql.connector

# Connect Database
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="boot",
    database="redbus"
)
cursor = db_connection.cursor()
print("Database connected Successfully")

# SQL to create table
create_table_query = """
CREATE TABLE IF NOT EXISTS BusDetails(
    id INT PRIMARY KEY AUTO_INCREMENT,
    route_name VARCHAR(255),
    route_link VARCHAR(255),
    bus_name VARCHAR(255),
    bus_type VARCHAR(255),
    departing_time TEXT,
    duration VARCHAR(50),
    reaching_time TEXT,
    star_rating TEXT,
    price VARCHAR(50),
    seat_availability TEXT
);
"""
cursor.execute(create_table_query)
print("Table created successfully")

# Initialize the WebDriver
driver = webdriver.Chrome()
driver.implicitly_wait(5)
wait = WebDriverWait(driver, 30)

# Navigate to the BSRTC Bus Routes page
driver.get('https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile')
driver.maximize_window()
time.sleep(2)

# Scrape route data
route_data = []
def scrape_routes():
    routes = driver.find_elements(By.XPATH, "//a[@class='route']")
    for route in routes:
        route_data.append({
            'Route Name': route.text,
            'Route Link': route.get_attribute('href')
        })
scrape_routes()
print("scraped routes successfully")

# Scroll to load more routes
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Pagination to collect routes from multiple pages
for page_number in range(1, 6):
    try:
        pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
        next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class,"DC_117_pageTabs") and text()="{page_number + 1}"]')

        actions = ActionChains(driver)
        actions.move_to_element(next_page_button).perform()
        next_page_button.click()
        time.sleep(1)

        wait.until(EC.text_to_be_present_in_element((By.XPATH, f'.//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        time.sleep(2)

        scrape_routes()

    except Exception as e:
        print(e)
        break

def collect_bus_details():
    alldetails = []
    for data in route_data:
        link = data['Route Link']
        route_name = data['Route Name']
        try:
            driver.get(link)
            time.sleep(2)

            # Click the 'View Buses' button if it exists
            try:
                # Wait for the button to be clickable
                view_buses_buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']")))
                
                for view_buses_button in view_buses_buttons:
                    wait.until(EC.element_to_be_clickable(view_buses_button)).click()
                    time.sleep(5)
            except Exception as e:
                print(e)

            # Scroll to load more buses
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            bus_containers = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'clearfix bus-item')]")))
            
            for bus_container in bus_containers:
                try:
                    busname = bus_container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bustype = bus_container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departuretiming = bus_container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = bus_container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reachtiming = bus_container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
                    rating = bus_container.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
                    price = bus_container.find_element(By.XPATH, ".//span[contains(@class,'f-19 f-bold') or contains(@class,'f-bold f-19')]").text
                    seat = bus_container.find_element(By.XPATH, ".//div[contains(@class, 'seat-left m-top-30') or contains(@class,'seat-left m-top-16')]").text
                    
                    details = {
                        'Route Name': route_name,
                        'Route Link': link,
                        'Bus Name': busname,
                        'Bus Type': bustype,
                        'Departing Time': departuretiming,
                        'Duration': duration,
                        'Reaching Time': reachtiming,
                        'Star Rating': rating,
                        'Price': price,
                        'Seat Availability': seat,
                    }

                    # Check if the details already exist in the list
                    if details not in alldetails:
                        print(details)
                        alldetails.append(details)

                        # Insert data into the database
                        insert_query = """
                        INSERT IGNORE INTO BusDetails (route_name, route_link, bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seat_availability)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        """
                        cursor.execute(insert_query, (
                            details['Route Name'],
                            details['Route Link'],
                            details['Bus Name'],
                            details['Bus Type'],
                            details['Departing Time'],
                            details['Duration'],
                            details['Reaching Time'],
                            details['Star Rating'],
                            details['Price'],
                            details['Seat Availability']
                        ))
                        db_connection.commit()

                except Exception as e:
                    print(e)

        except Exception as e:
            print(e)
    return alldetails

# Collect bus details and save to CSV
route_details = collect_bus_details()
df = pd.DataFrame(route_details)
df.to_csv("Bihar.csv", index=False)

# Close the WebDriver
driver.quit()

# Close the Database Connection
cursor.close()
db_connection.close()

Database connected Successfully
Table created successfully
scraped routes successfully
Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6EE6038A5+3004357]
	(No symbol) [0x00007FF6EE299970]
	(No symbol) [0x00007FF6EE14582A]
	(No symbol) [0x00007FF6EE195B8E]
	(No symbol) [0x00007FF6EE195E7C]
	(No symbol) [0x00007FF6EE1DEC27]
	(No symbol) [0x00007FF6EE1BBC1F]
	(No symbol) [0x00007FF6EE1DBA4C]
	(No symbol) [0x00007FF6EE1BB983]
	(No symbol) [0x00007FF6EE187628]
	(No symbol) [0x00007FF6EE188791]
	GetHandleVerifier [0x00007FF6EE62A00D+3161901]
	GetHandleVerifier [0x00007FF6EE67E060+3506048]
	GetHandleVerifier [0x00007FF6EE67400D+3465005]
	GetHandleVerifier [0x00007FF6EE3F0EEB+830987]
	(No symbol) [0x00007FF6EE2A467F]
	(No symbol) [0x00007FF6EE2A09D4]
	(No symbol) [0x00007FF6EE2A0B6D]
	(No symbol) [0x00007FF6EE290149]
	BaseThreadInitThunk [0x00007FFD56D4259D+29]
	RtlUserThreadStart [0x00007FFD5868AF38+40]

Message: 

Message: 

Message: 

Message: 

Message: 

{'Route Name': 'Patna (Bihar) t

In [2]:
df

,Route Name,Route Link,Bus Name,Bus Type,Departing Time,Duration,Reaching Time,Star Rating,Price,Seat Availability
0,Patna (Bihar) to Motihari,https://www.redbus.in/bus-tickets/patna-to-mot...,Jai Mata Di,AC Seater (2+2),17:00,04h 10m,21:10,5.0,350,41 Seats available
1,Patna (Bihar) to Motihari,https://www.redbus.in/bus-tickets/patna-to-mot...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,04h 55m,03:15,2.9,550,54 Seats available
2,Patna (Bihar) to Motihari,https://www.redbus.in/bus-tickets/patna-to-mot...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:00,06h 00m,04:00,4.5,550,52 Seats available
3,Patna (Bihar) to Purnea,https://www.redbus.in/bus-tickets/patna-to-purnea,Sri Krishna Tours and Travels,A/C Seater / Sleeper (2+2),20:30,08h 50m,05:20,3.3,600,10 Seats available
4,Patna (Bihar) to Purnea,https://www.redbus.in/bus-tickets/patna-to-purnea,Aarush Krrish,A/C Seater / Sleeper (2+1),21:00,09h 00m,06:00,2.0,600,43 Seats available
...,...,...,...,...,...,...,...,...,...,...
75,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...,Kusum Travels and Logistics,A/C Seater / Sleeper (2+1),17:30,08h 15m,01:45,1.6,2380.95,40 Seats available
76,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...,Baba Khatushyam Travels Pvt Ltd,A/C Seater / Sleeper (3+1),19:00,08h 00m,03:00,1.6,1050,58 Seats available
77,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...,Shree Balajee Travels And Cargo,A/C Seater / Sleeper (2+1),17:00,08h 00m,01:00,1.3,1439,47 Seats available
78,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...,Shree Balajee Travels And Cargo,A/C Seater / Sleeper (2+2),20:00,07h 30m,03:30,1.3,1709,72 Seats available
